<a href="https://colab.research.google.com/github/leochartrand/IFT615/blob/main/NaiveBayes/BagOfWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Cette démonstration est tirée du cours "CS50’s Introduction to Artificial Intelligence with Python" donné à l'Université d'Harvard.*
https://cs50.harvard.edu/ai/2024/
https://creativecommons.org/licenses/by-nc-sa/4.0/

Comme toujours, on commence par importer les librairies:

In [1]:
import nltk
nltk.download('punkt')

Pour passer à la suite, il nous faut préparer une certaine fonction utilitaire. La suivante permet d'extraire tous les mots distincts d'un document (ligne) et retourne un *set* de ceux-ci. 

In [2]:
def extract_words(document):
    return set(
        word.lower() for word in nltk.word_tokenize(document)
        if any(c.isalpha() for c in word)
    )

On peut maintenant charger le jeu de données. Il s'agit de deux fichiers contenant des échantillions positifs et négatifs.

In [3]:
corpus = []
for filename in ["positives.txt", "negatives.txt"]:
    with open(filename) as f:
        corpus.append([
            extract_words(line)
            for line in f.read().splitlines()
        ])

positives, negatives = corpus

Créons maintenant un *set* de tous les mots possibles qu'on peut retrouver dans nos échantillons:

In [5]:
words = set()
for document in positives:
    words.update(document)
for document in negatives:
    words.update(document)

Nous avons besoin de quelques autres fonctions utilitaires. La suivante permet d'associer les mots dans les échantillons à une étiquette, et assembler le tout dans une structure de données qui pourra être interprétée lors de l'apprentissage.

In [6]:
def generate_features(documents, words, label):
    features = []
    for document in documents:
        features.append(({
            word: (word in document)
            for word in words
        }, label))
    return features

La fonction ci-dessous permet de faire appel à notre modèle entraîné pour classifier un mot (ou plusieurs) donné en entrée.

In [7]:
def classify(classifier, document, words):
    document_words = extract_words(document)
    features = {
        word: (word in document_words)
        for word in words
    }
    return classifier.prob_classify(features)

Utilisons maintenant la fonction `generate_features` pour associer des mots à une certaine étiquette. Dans ce cas-ci, les étiquettes sont "Positive" et "Negative".

In [8]:
training = []
training.extend(generate_features(positives, words, "Positive"))
training.extend(generate_features(negatives, words, "Negative"))

Procédons maintenant à l'entrainement. Ici, la librairie NLTK fournit un classifieur déjà codé qu'on peut utiliser pour interpréter la structure de données qu'on vient de générer.

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training)

L'entraînement terminé, essayons de donner un mot à notre classifieur pour voir le résultat:

In [14]:
s = input("s: ")
result = (classify(classifier, s, words))
for key in result.samples():
    print(f"{key}: {result.prob(key):.4f}")

Positive: 0.5529
Negative: 0.4471


On peut voir qu'en donnant un mot connu par le modèle, on obtient une certaine probabilité qui penche vers le positif ou négatif. Cependant, si un mot ne se trouve pas dans le jeu de données d'entraînement, le modèle est indécis!